fine no preprocessing though

In [12]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class HandwritingIdentifier:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.sift = cv2.SIFT_create()
        self.orb = cv2.ORB_create()
        self.sift_classifier = SVC(kernel='rbf', probability=True)
        self.orb_classifier = SVC(kernel='rbf', probability=True)
        self.performance_metrics = {'sift': {}, 'orb': {}}

    def extract_features(self, img, method='sift'):
        if len(img.shape) > 2:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if method == 'sift':
            _, descriptors = self.sift.detectAndCompute(img, None)
        elif method == 'orb':
            _, descriptors = self.orb.detectAndCompute(img, None)
        else:
            raise ValueError("Invalid feature extraction method")
        if descriptors is None:
            return np.zeros(500)
        descriptors = descriptors.flatten()
        return descriptors[:500] if len(descriptors) > 500 else np.pad(descriptors, (0, 500 - len(descriptors)), 'constant')

    def load_dataset(self):
        sift_features, orb_features = [], []
        sift_labels, orb_labels = [], []
        for user_id in os.listdir(self.dataset_path):
            user_path = os.path.join(self.dataset_path, user_id)
            if os.path.isdir(user_path):
                for image_file in os.listdir(user_path):
                    image_path = os.path.join(user_path, image_file)
                    img = cv2.imread(image_path)
                    if img is not None:
                        sift_features.append(self.extract_features(img, 'sift'))
                        orb_features.append(self.extract_features(img, 'orb'))
                        sift_labels.append(user_id)
                        orb_labels.append(user_id)
        return {
            'sift': {'features': np.array(sift_features), 'labels': np.array(sift_labels)},
            'orb': {'features': np.array(orb_features), 'labels': np.array(orb_labels)},
        }

    def preprocess_features(self, features):
        scaler = StandardScaler()
        return scaler.fit_transform(features)

    def train_and_evaluate(self, features, labels, method):
        features = self.preprocess_features(features)
        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
        classifier = self.sift_classifier if method == 'sift' else self.orb_classifier
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred, average='weighted', zero_division=1),
            'confusion_matrix': confusion_matrix(y_test, y_pred).tolist(),
            'classification_report': classification_report(y_test, y_pred, zero_division=1),
        }
        return metrics

    def visualize_performance(self):
        methods = list(self.performance_metrics.keys())
        accuracies = [self.performance_metrics[m]['accuracy'] for m in methods]
        plt.bar(methods, accuracies)
        plt.title('Accuracy Comparison')
        plt.ylabel('Accuracy')
        plt.savefig('feature_comparison_metrics.png')
        plt.close()

def compare_feature_detection(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    sift_kp = sift.detect(gray, None)
    orb = cv2.ORB_create()
    orb_kp = orb.detect(gray, None)
    plt.figure(figsize=(15, 5))
    sift_img = cv2.drawKeypoints(gray, sift_kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    orb_img = cv2.drawKeypoints(gray, orb_kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.subplot(121), plt.imshow(cv2.cvtColor(sift_img, cv2.COLOR_BGR2RGB)), plt.title('SIFT Keypoints')
    plt.subplot(122), plt.imshow(cv2.cvtColor(orb_img, cv2.COLOR_BGR2RGB)), plt.title('ORB Keypoints')
    plt.tight_layout()
    plt.savefig('feature_comparison.png')
    plt.close()

def main():
    dataset_path = 'isolated_words_per_user'
    identifier = HandwritingIdentifier(dataset_path)
    dataset = identifier.load_dataset()
    for method in ['sift', 'orb']:
        print(f"\nEvaluating {method.upper()} Features:")
        metrics = identifier.train_and_evaluate(dataset[method]['features'], dataset[method]['labels'], method)
        identifier.performance_metrics[method] = metrics
        print(f"{method.upper()} Accuracy: {metrics['accuracy']:.4f}")
        print(f"Balanced Accuracy: {metrics['balanced_accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1_score']:.4f}")
        print("Classification Report:\n", metrics['classification_report'])
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
    identifier.visualize_performance()
    compare_feature_detection(r'isolated_words_per_user\user001\user001_abjadiyah_031.png')

if __name__ == '__main__':
    main()



Evaluating SIFT Features:
SIFT Accuracy: 0.0540
Balanced Accuracy: 0.0584
F1 Score: 0.4759
Classification Report:
               precision    recall  f1-score   support

     user001       0.25      0.04      0.07        26
     user002       0.17      0.10      0.12        20
     user003       0.05      0.19      0.07        16
     user004       0.04      0.05      0.04        20
     user005       1.00      0.00      0.00        25
     user006       0.00      0.00      1.00        25
     user007       0.00      0.00      1.00        19
     user008       0.07      0.04      0.05        25
     user009       0.06      0.35      0.11        17
     user010       0.00      0.00      1.00        13
     user011       0.00      0.00      1.00        30
     user012       0.00      0.00      1.00        22
     user013       0.05      0.07      0.05        15
     user014       0.08      0.05      0.06        20
     user015       0.10      0.04      0.06        23
     user016       

with preprocessing 

In [13]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class HandwritingIdentifier:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.sift = cv2.SIFT_create()
        self.orb = cv2.ORB_create()
        self.sift_classifier = SVC(kernel='rbf', probability=True)
        self.orb_classifier = SVC(kernel='rbf', probability=True)
        self.performance_metrics = {'sift': {}, 'orb': {}}

    def preprocess_image(self, img):
        """Preprocess the image for better feature extraction (e.g., binarization, noise removal)."""
        if len(img.shape) > 2:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian Blur
        blurred = cv2.GaussianBlur(img, (5, 5), 0)

        # Binarization using Otsu's method
        _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Morphological operations to remove noise
        kernel = np.ones((3, 3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)

        # Resize the image to a fixed size (128x128)
        normalized = cv2.resize(cleaned, (128, 128), interpolation=cv2.INTER_AREA)

        return normalized

    def extract_features(self, img, method='sift'):
        """Extract features using SIFT or ORB after preprocessing."""
        img = self.preprocess_image(img)

        if method == 'sift':
            _, descriptors = self.sift.detectAndCompute(img, None)
        elif method == 'orb':
            _, descriptors = self.orb.detectAndCompute(img, None)
        else:
            raise ValueError("Invalid feature extraction method")
        
        if descriptors is None:
            return np.zeros(500)
        descriptors = descriptors.flatten()
        return descriptors[:500] if len(descriptors) > 500 else np.pad(descriptors, (0, 500 - len(descriptors)), 'constant')

    def load_dataset(self):
        sift_features, orb_features = [], []
        sift_labels, orb_labels = [], []
        for user_id in os.listdir(self.dataset_path):
            user_path = os.path.join(self.dataset_path, user_id)
            if os.path.isdir(user_path):
                for image_file in os.listdir(user_path):
                    image_path = os.path.join(user_path, image_file)
                    img = cv2.imread(image_path)
                    if img is not None:
                        sift_features.append(self.extract_features(img, 'sift'))
                        orb_features.append(self.extract_features(img, 'orb'))
                        sift_labels.append(user_id)
                        orb_labels.append(user_id)
        return {
            'sift': {'features': np.array(sift_features), 'labels': np.array(sift_labels)},
            'orb': {'features': np.array(orb_features), 'labels': np.array(orb_labels)},
        }

    def preprocess_features(self, features):
        scaler = StandardScaler()
        return scaler.fit_transform(features)

    def train_and_evaluate(self, features, labels, method):
        features = self.preprocess_features(features)
        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
        classifier = self.sift_classifier if method == 'sift' else self.orb_classifier
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred, average='weighted', zero_division=1),
            'confusion_matrix': confusion_matrix(y_test, y_pred).tolist(),
            'classification_report': classification_report(y_test, y_pred, zero_division=1),
        }
        return metrics

    def visualize_performance(self):
        methods = list(self.performance_metrics.keys())
        accuracies = [self.performance_metrics[m]['accuracy'] for m in methods]
        plt.bar(methods, accuracies)
        plt.title('Accuracy Comparison')
        plt.ylabel('Accuracy')
        plt.savefig('feature_comparison_metrics.png')
        plt.close()

def compare_feature_detection(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    sift_kp = sift.detect(gray, None)
    orb = cv2.ORB_create()
    orb_kp = orb.detect(gray, None)
    plt.figure(figsize=(15, 5))
    sift_img = cv2.drawKeypoints(gray, sift_kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    orb_img = cv2.drawKeypoints(gray, orb_kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.subplot(121), plt.imshow(cv2.cvtColor(sift_img, cv2.COLOR_BGR2RGB)), plt.title('SIFT Keypoints')
    plt.subplot(122), plt.imshow(cv2.cvtColor(orb_img, cv2.COLOR_BGR2RGB)), plt.title('ORB Keypoints')
    plt.tight_layout()
    plt.savefig('feature_comparison.png')
    plt.close()

def main():
    dataset_path = 'isolated_words_per_user'
    identifier = HandwritingIdentifier(dataset_path)
    dataset = identifier.load_dataset()
    for method in ['sift', 'orb']:
        print(f"\nEvaluating {method.upper()} Features:")
        metrics = identifier.train_and_evaluate(dataset[method]['features'], dataset[method]['labels'], method)
        identifier.performance_metrics[method] = metrics
        print(f"{method.upper()} Accuracy: {metrics['accuracy']:.4f}")
        print(f"Balanced Accuracy: {metrics['balanced_accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1_score']:.4f}")
        print("Classification Report:\n", metrics['classification_report'])
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
    identifier.visualize_performance()
    compare_feature_detection(r'isolated_words_per_user\user001\user001_abjadiyah_031.png')

if __name__ == '__main__':
    main()



Evaluating SIFT Features:
SIFT Accuracy: 0.0417
Balanced Accuracy: 0.0455
F1 Score: 0.5945
Classification Report:
               precision    recall  f1-score   support

     user001       0.20      0.04      0.06        26
     user002       0.00      0.00      1.00        20
     user003       0.03      0.12      0.05        16
     user004       0.00      0.00      1.00        20
     user005       0.00      0.00      1.00        25
     user006       0.00      0.00      1.00        25
     user007       0.11      0.05      0.07        19
     user008       0.11      0.04      0.06        25
     user009       0.11      0.29      0.16        17
     user010       0.00      0.00      1.00        13
     user011       0.00      0.00      1.00        30
     user012       0.03      0.05      0.04        22
     user013       0.00      0.00      1.00        15
     user014       0.04      0.05      0.04        20
     user015       0.00      0.00      1.00        23
     user016       

extra stuff

In [14]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    make_scorer,
)
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

class HandwritingIdentifier:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.sift = cv2.SIFT_create()
        self.orb = cv2.ORB_create()
        self.performance_metrics = {'sift': {}, 'orb': {}}

    def preprocess_image(self, img):
        """Preprocess the image for better feature extraction (e.g., binarization, noise removal)."""
        if len(img.shape) > 2:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian Blur
        blurred = cv2.GaussianBlur(img, (5, 5), 0)

        # Binarization using Otsu's method
        _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Morphological operations to remove noise
        kernel = np.ones((3, 3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)

        # Resize the image to a fixed size (128x128)
        normalized = cv2.resize(cleaned, (128, 128), interpolation=cv2.INTER_AREA)

        return normalized

    def extract_features(self, img, method='sift'):
        """Extract features using SIFT or ORB after preprocessing."""
        img = self.preprocess_image(img)

        if method == 'sift':
            _, descriptors = self.sift.detectAndCompute(img, None)
        elif method == 'orb':
            _, descriptors = self.orb.detectAndCompute(img, None)
        else:
            raise ValueError("Invalid feature extraction method")
        
        if descriptors is None:
            return np.zeros(500)
        descriptors = descriptors.flatten()
        return descriptors[:500] if len(descriptors) > 500 else np.pad(descriptors, (0, 500 - len(descriptors)), 'constant')

    def load_dataset(self):
        sift_features, orb_features = [], []
        sift_labels, orb_labels = [], []
        for user_id in os.listdir(self.dataset_path):
            user_path = os.path.join(self.dataset_path, user_id)
            if os.path.isdir(user_path):
                for image_file in os.listdir(user_path):
                    image_path = os.path.join(user_path, image_file)
                    img = cv2.imread(image_path)
                    if img is not None:
                        sift_features.append(self.extract_features(img, 'sift'))
                        orb_features.append(self.extract_features(img, 'orb'))
                        sift_labels.append(user_id)
                        orb_labels.append(user_id)
        return {
            'sift': {'features': np.array(sift_features), 'labels': np.array(sift_labels)},
            'orb': {'features': np.array(orb_features), 'labels': np.array(orb_labels)},
        }

    def preprocess_features(self, features):
        """Standardize features and apply PCA for dimensionality reduction."""
        scaler = StandardScaler()
        standardized_features = scaler.fit_transform(features)
        pca = PCA(n_components=100)
        return pca.fit_transform(standardized_features)

    def hyperparameter_tuning(self, X, y):
        """Perform hyperparameter tuning using GridSearchCV."""
        param_grid = {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1], 'kernel': ['rbf']}
        grid = GridSearchCV(SVC(probability=True), param_grid, cv=5, scoring='accuracy', verbose=2)
        grid.fit(X, y)
        return grid.best_estimator_

    def train_and_evaluate(self, features, labels, method):
        features = self.preprocess_features(features)
        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

        # Hyperparameter tuning
        classifier = self.hyperparameter_tuning(X_train, y_train)

        # k-fold cross-validation
        cross_val_scores = cross_val_score(classifier, X_train, y_train, cv=5)

        # Train and evaluate
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        y_prob = classifier.predict_proba(X_test)[:, 1] if len(np.unique(y_test)) == 2 else None

        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred, average='weighted', zero_division=1),
            'precision': precision_score(y_test, y_pred, average='weighted', zero_division=1),
            'recall': recall_score(y_test, y_pred, average='weighted', zero_division=1),
            'roc_auc': roc_auc_score(y_test, y_prob, average='weighted') if y_prob is not None else None,
            'confusion_matrix': confusion_matrix(y_test, y_pred).tolist(),
            'classification_report': classification_report(y_test, y_pred, zero_division=1),
            'cross_val_mean': np.mean(cross_val_scores),
        }
        return metrics

    def visualize_performance(self):
        methods = list(self.performance_metrics.keys())
        accuracies = [self.performance_metrics[m]['accuracy'] for m in methods]
        plt.bar(methods, accuracies)
        plt.title('Accuracy Comparison')
        plt.ylabel('Accuracy')
        plt.savefig('feature_comparison_metrics.png')
        plt.close()

def main():
    dataset_path = 'isolated_words_per_user'
    identifier = HandwritingIdentifier(dataset_path)
    dataset = identifier.load_dataset()
    for method in ['sift', 'orb']:
        print(f"\nEvaluating {method.upper()} Features:")
        metrics = identifier.train_and_evaluate(dataset[method]['features'], dataset[method]['labels'], method)
        identifier.performance_metrics[method] = metrics
        for key, value in metrics.items():
            if key != 'confusion_matrix' and key != 'classification_report':
                print(f"{method.upper()} {key.replace('_', ' ').capitalize()}: {value:.4f}" if value else f"{key}: N/A")
        print("Classification Report:\n", metrics['classification_report'])
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
    identifier.visualize_performance()

if __name__ == '__main__':
    main()



Evaluating SIFT Features:
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  26.0s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  21.3s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  19.1s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  19.7s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  16.7s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  17.3s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  16.4s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  15.9s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  16.7s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  16.2s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  16.4s
[CV] END .............